# Project: Wrangling and Analyze Data
#### Arnd-Olav Feindt
---

## Table of contents <a class="anchor" id="Tableofcontents"></a>

- [Data Gathering](#DataGathering)
- [Assessing Data](#AssessingData)
  - [Table overview](#Tableoverview)
  - [All found issues](#Allfoundissues)
  - [Quality issues](#Qualityissues)
  - [Tidiness issues](#Tidinessissues)
- [Cleaning Data](#CleaningData)
- [Storing Data](#StoringData)
- [Analyzing and Visualizing Data](#AnalyzingandVisualizingData)
  - [Visualization](#Visualization)
  - [Insights:](#Insights)
  
---

## Data Gathering <a class="anchor" id="DataGathering"></a>
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os
import requests
import tweepy
import json
from timeit import default_timer as timer
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_object_dtype
from IPython.display import display, Markdown  # To write (have) code and markdown in the same cell


In [2]:
# Load csv into dataframe
wrd_archive = pd.read_csv('twitter-archive-enhanced.csv')

# View head of dataframe
wrd_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [3]:
# Fetch tsv file from Udactiy server
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(url.split('/')[-1], mode='wb') as file:
    file.write(response.content)
    
# Load downloaded tsv file into dataframe
wrd_image_prediction = pd.read_csv('image-predictions.tsv', sep = '\t')

# View head of dataframe
wrd_image_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [4]:
# Fetch the data from Twitter if you haven't done it already... because it can take 20 - 30 minutes
file_name = 'tweet_json.txt'
if not os.path.isfile(file_name):
    
    consumer_key = 'jK#######################JQ'
    consumer_secret = 'QZ#######################MX'
    access_token = '62#######################oJ'
    access_secret = 'Gr#######################nZ'

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

    # Getting tweet details for a single tweet_id from the twitter archive
    tweet = api.get_status(	892420643555336193, tweet_mode = 'extended' )

    # Viewing the json data
    tweet._json

    # Verifying the presence of retweet count of tweet_id in json data
    tweet._json['retweet_count']

    # Verifying the presence of favorite count of tweet_id in json data
    tweet._json['favorite_count']

    # Creating file
    open(file_name, 'w').close()

    # Query Twitter's API for JSON data for each tweet ID in the Twitter archive & writing twitter json data to tweet_json.txt file
    tweet_id_errors = {}
    count = 0
    start = timer()
    with open(file_name, mode = 'w') as file:
        for tweet_id in wrd_archive.tweet_id:
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                file.write(json.dumps(tweet._json) + '\n')
                print(str(count) + '_' + str(tweet_id))
                count +=1
            except Exception as e:
                print(str(count) + '_' + str(tweet_id) + str(e))
                tweet_id_errors[str(count) + '_' + str(tweet_id)] = tweet._json
                count +=1
    end = timer()
    print('Elapsed Time:' + str(end-start))

In [5]:
# Reading twitter json data from tweet_json.txt file by converting each json string into python dictionary
# and appending them to a list (row by row) and this list of dictionaries will eventually be converted to a pandas DataFrame 

df_list = []
with open('tweet_json.txt', mode = 'r') as file:
    for row in file.readlines():
        tweet_dict = json.loads(row)
        retweet_count = tweet_dict['retweet_count']
        favorite_count = tweet_dict['favorite_count']
        tweet_id = tweet_dict['id']
        df_list.append({'tweet_id': tweet_id, 
                        'retweet_count': retweet_count, 
                        'favorite_count': favorite_count})  

# Creating DataFrame from list of dictionaries
wrd_archive_add = pd.DataFrame(df_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])

# Top five rows of dataframe
wrd_archive_add.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,6975,33711
1,892177421306343426,5276,29231
2,891815181378084864,3465,21982
3,891689557279858688,7196,36804
4,891327558926688256,7721,35192


[top](#Tableofcontents)

## Assessing Data <a class="anchor" id="AssessingData"></a>
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [6]:
def visual(dataframe, string_name):
    display(Markdown("##### A. Head from " + string_name))
    print(dataframe.head())
    display(Markdown("##### B. Tail from " + string_name))
    print(dataframe.tail())
    display(Markdown("##### C. Sample of 25 rows from " + string_name))
    df_elements = dataframe.sample(n=25)
    print(df_elements.head(25))

In [18]:
def programmatic(dataframe, string_name):
    display(Markdown(f"##### D. Shape from {string_name}:"))
    print('')
    print(dataframe.shape)
    display(Markdown(f"##### E. Info from {string_name}:"))
    print('')

    print(dataframe.info())
    display(Markdown(f"##### F. isnull sum from {string_name}:"))
    print('')
    print(dataframe.isnull().sum())
    display(Markdown(f"##### G. Duplicates from {string_name}:"))
    print('')
    print(sum(dataframe.duplicated()))
    display(Markdown(f"##### H. Describe from {string_name}:"))
    print('')
    print(dataframe.describe())
    display(Markdown(f"##### I. Nunique from {string_name}:"))
    print('')
    print(dataframe.nunique())

    display(Markdown(f"##### J. Duplicates per column from {string_name}:"))
    for i, col in zip(dataframe.dtypes, dataframe):
        print('')
        print(
            f"Sum of duplicates in {col} = "
            + str(sum(dataframe[col].duplicated()))
            + f"({i})"
        )

    display(Markdown(f"##### K. Sorted numeric values per column in {string_name}:"))
    for i, col in zip(dataframe.dtypes, dataframe):
        print('')
        print(f"Sorted numeric values per column in {col}")

        if is_numeric_dtype(
            dataframe[col]
        ):
            print(np.sort(dataframe[col].unique()))
        else:
            print(col + " is not numeric")

    display(Markdown(f"##### L. Check which column is object type from {string_name}:"))
    for i, col in zip(dataframe.dtypes, dataframe):
        print('')
        if is_object_dtype(dataframe[col]):
            print(str(col), 'is', str(i))
        else:
            print(str(col), 'is not')

    display(Markdown(f"##### M. Count different values per column from {string_name}:"))
    
    for i, col in zip(dataframe.dtypes, dataframe):
        print('')
        print(str(col))
        print(dataframe[col].value_counts())
        print('')


In [19]:
dfs = {'wrd_archive' : wrd_archive,
       'wrd_image_prediction' : wrd_image_prediction,
       'wrd_archive_add':wrd_archive_add
      }
print('__________________________')

for k, v in dfs.items():
    display(Markdown(f'### {k} - Visual assessment'))
    visual(v, k)
    display(Markdown(f'### {k} - Programmatic assessment'))
    programmatic(v, k)
    print ('')

__________________________


### wrd_archive - Visual assessment

##### A. Head from wrd_archive

             tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  r

##### B. Tail from wrd_archive

...


Sorted numeric values per column in tweet_id
[666020888022790149 666029285002620928 666033412701032449 ...
 891815181378084864 892177421306343426 892420643555336193]

Sorted numeric values per column in retweet_count
[    1     2     3 ... 51452 52659 70366]

Sorted numeric values per column in favorite_count
[     0     45     59 ... 111241 123716 144304]


##### L. Check which column is object type from wrd_archive_add:


tweet_id is not

retweet_count is not

favorite_count is not


##### M. Count different values per column from wrd_archive_add:


tweet_id
892420643555336193    1
688828561667567616    1
687460506001633280    1
687399393394311168    1
687317306314240000    1
                     ..
772877495989305348    1
772826264096874500    1
772615324260794368    1
772581559778025472    1
666020888022790149    1
Name: tweet_id, Length: 2327, dtype: int64


retweet_count
819     5
521     5
406     5
50      5
669     5
       ..
3614    1
2707    1
4261    1
2328    1
420     1
Name: retweet_count, Length: 1633, dtype: int64


favorite_count
0        160
1148       4
371        4
1275       3
657        3
        ... 
4724       1
3919       1
13356      1
5070       1
2285       1
Name: favorite_count, Length: 1971, dtype: int64




In [13]:
num

NameError: name 'num' is not defined

[top](#Tableofcontents)

### Table overview <a class="anchor" id="Tableoverview"></a>
 - **wrd_archive - contains twitter archive data of WeRateDogs**
 - **wrd_image_prediction - contains twitter api data (retweets and likes) of WeRateDogs**
 - **wrd_archive_add - contains tweet image prediction data**
---
#### What should the type and possible values of the columns be
##### wrd_archive
|Column|actual type|target type | possible values|
|----------------------------|-------|----------|-------|
| tweet_id                   |  int64 | string | 1 != empty  |
| in_reply_to_status_id      |  float64 | int64 |   |
| in_reply_to_user_id        |  float64 | int64 |   |
| timestamp                  |  object | Timestamp |   |
| source                     |  object | string |   |
| text                       |  object | string |   |
| retweeted_status_id        | float64  | int64 |   |
| retweeted_status_user_id   | float64  | int64 |   |
| retweeted_status_timestamp | object  | int64 |   |
| expanded_urls              |  object | string |   |
| rating_numerator           | int64  | int64 |   |
| rating_denominator         |  int64 | int64 |   |
| name                       | object  | string |   |
| doggo                      |  object | string |   |
| floofer                    | object  | string |   |
| pupper                     | object  | string |   |
| puppo                      | object  | string |    |

##### wrd_image_prediction
|Column|actual type|target type | possible values|
|----------|-----------------------|---|---| 
| tweet_id  |int64 | string  |   |
| jpg_url  |object | string  |   |
| img_num  | int64| int64   |   |
| p1       |object | string  |   |
| p1_conf  | float64| float64 |   |
| p1_dog   |bool | bool    |   |
| p2       | object| string  |   |
| p2_conf  | float64| float64 |   |
| p2_dog   | bool| bool    |   |
| p3       |object | string  |   |
| p3_conf  |float64 | float64 |   |
| p3_dog   | bool| bool    |   |

##### wrd_archive_add
|Column|actual type|target type | possible values|
|----------|-----------------------|---|---| 
| tweet_id  | int64 | string  |   |
| retweet_count  | int64 | int64   |   |
| favorite_count | int64 | int64  |   |

### All found issues <a class="anchor" id="Allfoundissues"></a>

| **Issue ID** | **Issue Type** | **Method** | **Dimension** | **Table** | **Column** | **Description** |
|---|---|---|---|---|---|---|
| 1 | Q | P | Completeness | wrd_archive | in_reply_to_status_id<br>in_reply_to_user_id | Observations for Uninterested reply columns present(78). |
| 2 | Q | P | Completeness | wrd_archive | retweeted_status_timestamp<br>retweeted_status_id<br>retweeted_status_user_id | Observations for Uninterested retweet columns present(181). |
| 3 | Q | V | Completeness | wrd_archive | in_reply_to_status_id<br>in_reply_to_user_id<br>retweeted_status_timestamp<br>retweeted_status_id<br>retweeted_status_user_id | Unnecessary columns |
| 4 | Q | P | Validity | wrd_archive<br>wrd_image_prediction<br>wrd_archive_add | tweet_id | Type *int64* instead of *string* |
| 5 | Q | P | Validity | wrd_archive | timestamp<br>retweeted_status_timestamp | Type *object* instead of *timestamp* |
| 6 | Q | V | Accuracy | wrd_archive | rating_numerator | Inaccurate values |
| 7 | Q | V | Accuracy | wrd_archive | rating_denominator | Inaccurate values != 10 |
| 8 | Q | V | Consistency | wrd_archive | rating_numerator<br>rating_denominator | Inconsistent rating |
| 9 | Q | P | Consistency | wrd_archive | expanded_urls | Sometimes more than one value |
| 10 | Q | V | Validity | wrd_archive | name | Invalid names or non-names |
| 11 | Q | P | Consistency | wrd_archive | text | ‘&’ and ‘\n’ in (sub)strings |
| 12 | Q | V | Consistency | wrd_archive | doggo<br>floofer<br>pupper<br>puppo | Categorical variables represented as separate column. |
| 13 | Q | V | Validity | wrd_image_prediction | p1_conf, p2_conf, p3_conf | Drop the rows with FALSE in all these columns |
| 14 | Q | V | Consistency | wrd_image_prediction | p1<br>p2<br>p3 | One Prediction data with TRUE and higher confidence is enough among the top three predictions. |
| 15 | T | P | - | wrd_archive | source | Multiple data values in one column (HTML tags URL and content) |
| 16 | T | P | - | wrd_archive | text | Multiple data values in one column (Text and URL) |
| 17 | T | P | - | wrd_archive<br>wrd_image_prediction<br>wrd_archive_add | tweet_id | Duplicate column in three DataFrame |
| 18 | T | V | - | wrd_image_prediction | p1, p1_dog, p1_conf<br>p2, p2_dog, p2_conf<br>p3, p3_dog, p3_conf | For quality I will create two new columns for image prediction and confidence level. After I will drop p1, p2, p3, p1_conf, p2_conf, p3_conf, p1_dog, p2_conf and p3_conf |
| 19 | Q | V | Consistency | wrd_image_prediction | p1<br>p2<br>p3 | Replace '_' with space in breed and Replace '-' with space/no space for breeds wherever needed based on actual name browsing |
| 20 | T | V | - | wrd_image_prediction | breed | Capitalize the breed strings |

### Quality issues <a class="anchor" id="Qualityissues"></a>

I choose the issues 1,2,3,4,5,6,7 and 12

[top](#Tableofcontents)

### Tidiness issues <a class="anchor" id="Tidinessissues"></a>

I choose the issues 15, 16 and 18

[top](#Tableofcontents)

## Cleaning Data <a class="anchor" id="CleaningData"></a>
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

[top](#Tableofcontents)

In [32]:
# Make copies of original pieces of data
wrd_archive_clean = wrd_archive.copy()
wrd_image_prediction_clean = wrd_image_prediction.copy()
wrd_archive_add_clean = wrd_archive_add.copy()

### Issue 1-Q

#### Define
**wrd_archive_clean/in_reply_to_status_id, in_reply_to_user_id**	Not necessary observations of uninterested reply columns present. I will drop the rows with input in these columns.

#### Code

In [33]:
# After counting the affected rows I will delete them within the 'Test' block bellow
print(len(wrd_archive_clean.query('in_reply_to_status_id != "NaN"')))
print(len(wrd_archive_clean.query('in_reply_to_user_id != "NaN"')))

78
78


#### Test

In [34]:
print(wrd_archive_clean.shape)
wrd_archive_clean = wrd_archive_clean[~wrd_archive_clean['in_reply_to_status_id'].notna()]
print(wrd_archive_clean.shape)

(2356, 17)
(2278, 17)


### Issue 2-Q

#### Define
**wrd_archive_clean/retweeted_status_timestamp, retweeted_status_id, retweeted_status_user_id**

Not necessary observations of uninterested retweet columns present.
I will drop the rows with input in these columns.
I am facing a problem here with counting the NaN in column 'retweeted_status_timestamp', so I let it.

#### Code

In [35]:
# After counting the affected rows I will delete them within the 'Test' block bellow
print(len(wrd_archive_clean[~wrd_archive_clean['retweeted_status_id'].isnull()]))
print(len(wrd_archive_clean[~wrd_archive_clean['retweeted_status_user_id'].isnull()]))


181
181


#### Test

In [36]:
print(wrd_archive_clean.shape)
wrd_archive_clean = wrd_archive_clean[wrd_archive_clean['retweeted_status_id'].isnull()]
wrd_archive_clean = wrd_archive_clean[wrd_archive_clean['retweeted_status_user_id'].isnull()]
print(wrd_archive_clean.shape)

(2278, 17)
(2097, 17)


### Issue 3-Q

#### Define
**wrd_archive_clean/in_reply_to_status_id, in_reply_to_user_id, retweeted_status_timestamp, retweeted_status_id, retweeted_status_user_id**

Unnecessary columns.

I will drop these columns

#### Code

In [37]:
print(wrd_archive_clean.shape)
wrd_archive_clean.drop(['in_reply_to_status_id',
                        'in_reply_to_user_id',
                        'retweeted_status_timestamp',
                        'retweeted_status_id',
                        'retweeted_status_user_id'
                        ], axis=1, inplace=True)

(2097, 17)


#### Test

In [38]:
print(wrd_archive_clean.shape, "\n")
print(wrd_archive_clean.info())

(2097, 12) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB
None


### Issue 4-Q

#### Define
**wrd_archive, wrd_image_prediction, wrd_archive_add/tweet_id**	Type int64 instead of string
I will change the type of the columns to string.

#### Code

In [39]:
wrd_archive_clean.tweet_id = wrd_archive_clean.tweet_id.astype(str)
wrd_image_prediction_clean.tweet_id = wrd_image_prediction_clean.tweet_id.astype(str)
wrd_archive_add_clean.tweet_id = wrd_archive_add_clean.tweet_id.astype(str)

#### Test

In [40]:
print(wrd_archive_clean.tweet_id.apply(type), "\n")
print(wrd_image_prediction_clean.tweet_id.apply(type), "\n")
print(wrd_archive_add_clean.tweet_id.apply(type), "\n")

0       <class 'str'>
1       <class 'str'>
2       <class 'str'>
3       <class 'str'>
4       <class 'str'>
5       <class 'str'>
6       <class 'str'>
7       <class 'str'>
8       <class 'str'>
9       <class 'str'>
10      <class 'str'>
11      <class 'str'>
12      <class 'str'>
13      <class 'str'>
14      <class 'str'>
15      <class 'str'>
16      <class 'str'>
17      <class 'str'>
18      <class 'str'>
20      <class 'str'>
21      <class 'str'>
22      <class 'str'>
23      <class 'str'>
24      <class 'str'>
25      <class 'str'>
26      <class 'str'>
27      <class 'str'>
28      <class 'str'>
29      <class 'str'>
31      <class 'str'>
            ...      
2326    <class 'str'>
2327    <class 'str'>
2328    <class 'str'>
2329    <class 'str'>
2330    <class 'str'>
2331    <class 'str'>
2332    <class 'str'>
2333    <class 'str'>
2334    <class 'str'>
2335    <class 'str'>
2336    <class 'str'>
2337    <class 'str'>
2338    <class 'str'>
2339    <class 'str'>
2340    <c

### Issue 5-Q

#### Define
**wrd_archive_clean/timestamp, retweeted_status_timestamp (already dropped)** Type *object* instead of *timestamp*
I will change the column type to `timestamp`

#### Code

In [41]:
wrd_archive_clean.timestamp = pd.to_datetime(wrd_archive_clean.timestamp)

#### Test

In [42]:
print(wrd_archive_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null object
timestamp             2097 non-null datetime64[ns]
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 213.0+ KB
None


### Issue 6-Q

#### Define
**wrd_archive_clean/rating_denominator**
Inaccurate values != 10. As these are just 17 rows I will drop them.

#### Code

In [43]:
# After counting the affected rows I will delete them within the 'Test' block bellow
len(wrd_archive_clean.query('rating_denominator != 10'))

17

#### Test

In [44]:
print(wrd_archive_clean.shape)
wrd_archive_clean = wrd_archive_clean[wrd_archive_clean.rating_denominator == 10]
print(wrd_archive_clean.shape)

(2097, 12)
(2080, 12)


### Issue 7-Q

#### Define
**wrd_archive_clean/rating_numerator**

We have some numerator >= 15 rows, there is a big gap between 14 and 26. This seems unrealisted watching the distribution. As these are just 5 rows I will drop them.

#### Code

In [45]:
wrd_archive_clean['rating_numerator'].value_counts().sort_index(ascending = False)

1776      1
420       1
75        1
27        1
26        1
14       38
13      287
12      486
11      413
10      436
9       152
8        98
7        51
6        32
5        34
4        15
3        19
2         9
1         4
0         1
Name: rating_numerator, dtype: int64

#### Test

In [46]:
# After counting the affected rows I will delete them within the 'Test' block bellow
print(len(wrd_archive_clean.query('rating_numerator >= 15')),"\n")

print(wrd_archive_clean.shape)
wrd_archive_clean = wrd_archive_clean[wrd_archive_clean.rating_numerator <= 15]
print(wrd_archive_clean.shape)

5 

(2080, 12)
(2075, 12)


### Issue 12-Q

#### Define

**wrd_archive_clean/doggo, floofer, pupper, puppo**

Categorical variables represented as separate columns.

I will create a column dog_stage with the values from these columns and drop them afterwards.

#### Code

In [47]:
import re
wrd_archive_clean['dog_stage'] = wrd_archive_clean.apply(lambda x : re.sub('None,*','',(x.doggo +','+ x.floofer + x.puppo + x.pupper)).rstrip(',') 
                     if re.sub('None','',(x.doggo + x.floofer + x.pupper + x.puppo)) != ''
                     else 'None', axis=1)

columns_to_drop = ['doggo',
                   'floofer',
                   'pupper',
                   'puppo'
                  ]
wrd_archive_clean.drop(columns = columns_to_drop, inplace = True)


#### Test

In [48]:
wrd_archive_clean

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None
2,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None
3,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None
4,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None
5,891087950875897856,2017-07-29 00:08:17,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,https://twitter.com/dog_rates/status/891087950...,13,10,None,None
6,890971913173991426,2017-07-28 16:27:12,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None
7,890729181411237888,2017-07-28 00:22:40,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,https://twitter.com/dog_rates/status/890729181...,13,10,None,None
8,890609185150312448,2017-07-27 16:25:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None
9,890240255349198849,2017-07-26 15:59:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo


### Issue 7-Q

#### Define

**wrd_image_prediction_clean/p1, p1_dog, p1_conf, p2, p2_dog, p2_conf, p3, p3_dog, p3_conf**

For quality I will create two new columns for image prediction and confidence level. After I will drop p1, p2, p3, p1_conf, p2_conf, p3_conf, p1_dog, p2_conf and p3_conf

#### Define


#### Code

In [51]:
# Create a list to store the best prediction and confidence values
prediction_list = []

# Define a function to perform the extraction process
def extract_breed_info(row):
    """
        Extracts the best prediction and confidence value from passed row.
        Params:
            row: a row from the dataframe of interest.
        Output:
            A dictionary containing prediction and confidence appended into prediction list.
            Prints a status update of extraction process.
    """
    if row.p1_dog:
        prediction_list.append({'breed': row.p1,'confidence': row.p1_conf})
    elif row.p2_dog:
        prediction_list.append({'breed': row.p2,'confidence': row.p2_conf})
    elif row.p3_dog:
        prediction_list.append({'breed': row.p3,'confidence': row.p3_conf})
    else:
        prediction_list.append({'breed': 'Unknown','confidence': 0})
        
    return 'Info extracted to prediction list'

In [53]:
# Run the extraction process
wrd_image_prediction_clean.apply(extract_breed_info, axis=1)

0       Info extracted to prediction list
1       Info extracted to prediction list
2       Info extracted to prediction list
3       Info extracted to prediction list
4       Info extracted to prediction list
5       Info extracted to prediction list
6       Info extracted to prediction list
7       Info extracted to prediction list
8       Info extracted to prediction list
9       Info extracted to prediction list
10      Info extracted to prediction list
11      Info extracted to prediction list
12      Info extracted to prediction list
13      Info extracted to prediction list
14      Info extracted to prediction list
15      Info extracted to prediction list
16      Info extracted to prediction list
17      Info extracted to prediction list
18      Info extracted to prediction list
19      Info extracted to prediction list
20      Info extracted to prediction list
21      Info extracted to prediction list
22      Info extracted to prediction list
23      Info extracted to predicti

In [56]:
# Assign the values in prediction list into new columns in predictions_clean
wrd_image_prediction_clean[['breed', 'confidence']] = pd.DataFrame(prediction_list)

# Round confidence to three decimal places
wrd_image_prediction_clean.confidence = round(wrd_image_prediction_clean.confidence, 3)

In [57]:
# Verify the extraction process
wrd_image_prediction_clean.iloc[:, 3:].sample(5)

,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,breed,confidence
635,Irish_setter,0.406047,True,cocker_spaniel,0.345646,True,Airedale,1.479120e-01,True,Irish_setter,0.406
1748,dingo,0.280949,False,German_shepherd,0.194044,True,Pembroke,1.200510e-01,True,German_shepherd,0.194
119,Chihuahua,0.664834,True,cowboy_boot,0.060343,False,giant_panda,5.983750e-02,False,Chihuahua,0.665
1556,doormat,0.874431,False,French_bulldog,0.018759,True,Boston_bull,1.513440e-02,True,French_bulldog,0.019
1760,shopping_cart,0.995941,False,shopping_basket,0.004057,False,mousetrap,8.832830e-07,False,Unknown,0.000


In [87]:
#create new columns
wrd_image_prediction_clean['dog_breed'] = dog_breed
wrd_image_prediction_clean['confidence_level'] = confidence_level

ValueError: Length of values does not match length of index

In [58]:
# Create a list of unwanted columns
unwanted_columns = ['p1','p2', 'p3','p1_conf', 'p2_conf', 'p3_conf','p1_dog', 'p2_dog', 'p3_dog', 'img_num']

# Drop all unwanted columns
wrd_image_prediction_clean.drop(columns=unwanted_columns, inplace=True)

#### Test

In [60]:
wrd_image_prediction_clean.head(5)

,tweet_id,jpg_url,breed,confidence
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel,0.465
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,redbone,0.507
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German_shepherd,0.596
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian_ridgeback,0.408
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,miniature_pinscher,0.560


### Issue 8-Q

#### Define:
**wrd_image_prediction_clean/dog_breed**

Replace '_' with space in breed and Replace '-' with space/no space for breeds wherever needed based on actual name browsing

#### Code

In [61]:
# Remove all underscores and format the breed text to titlecase.
wrd_image_prediction_clean.breed = wrd_image_prediction_clean.breed.str.replace('_', ' ').str.title()

#### Test

In [62]:
wrd_image_prediction_clean.breed.unique()


array(['Welsh Springer Spaniel', 'Redbone', 'German Shepherd',
       'Rhodesian Ridgeback', 'Miniature Pinscher', 'Bernese Mountain Dog',
       'Unknown', 'Chow', 'Golden Retriever', 'Miniature Poodle',
       'Gordon Setter', 'Walker Hound', 'Pug', 'Bloodhound', 'Lhasa',
       'English Setter', 'Italian Greyhound', 'Maltese Dog',
       'Newfoundland', 'Malamute', 'Soft-Coated Wheaten Terrier',
       'Chihuahua', 'Black-And-Tan Coonhound', 'Toy Terrier',
       'Blenheim Spaniel', 'Pembroke', 'Irish Terrier',
       'Chesapeake Bay Retriever', 'Curly-Coated Retriever', 'Dalmatian',
       'Ibizan Hound', 'Border Collie', 'Labrador Retriever',
       'Miniature Schnauzer', 'Airedale', 'Rottweiler',
       'West Highland White Terrier', 'Toy Poodle', 'Giant Schnauzer',
       'Vizsla', 'Siberian Husky', 'Papillon', 'Saint Bernard',
       'Tibetan Terrier', 'Borzoi', 'Beagle', 'Yorkshire Terrier',
       'Pomeranian', 'Kuvasz', 'Flat-Coated Retriever',
       'Norwegian Elkhound', '

### Issue 18-T

#### Define:
**wrd_image_prediction/**

Merge all DataFrames into one Master DataFrame

#### Code

In [63]:
# Merge archive clean and prediction clean into master df
master_df = pd.merge(wrd_archive_clean, wrd_image_prediction, on='tweet_id', how='inner')

# Merge json clean into master df
master_df = pd.merge(master_df, wrd_archive_add, on='tweet_id', how='inner')

# Check results 
master_df.info()

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

#### Test

[top](#Tableofcontents)

## Storing Data <a class="anchor" id="StoringData"></a>
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

[top](#Tableofcontents)

## Analyzing and Visualizing Data <a class="anchor" id="AnalyzingandVisualizingData"></a>
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

[top](#Tableofcontents)

### Insights: <a class="anchor" id="Insights"></a>
1.

2.

3.

[top](#Tableofcontents)

### Visualization <a class="anchor" id="Visualization"></a>

[top](#Tableofcontents)